##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Analyzer

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TensorFlow Lite Model Analyzer API を使用すると、モデルの構造をリスト出力することで、TensorFlow Lite 形式でモデルを分析できます。


## Model Analyzer API

次の API は TensorFlow Lite Model Analyzer で使用できます。

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

API の詳細については、https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer をご覧ください。Python ターミナルから `help(tf.lite.experimental.Analyzer.analyze)` を実行することもできます。


## シンプルな Keras モデルを利用した基本的な使用方法

次のコードでは Model Analyzer の基本的な使用方法を示します。TFLite モデルコンテンツの変換された Keras モデルのコンテンツです。flatbuffer オブジェクトの形式です。

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


## MobileNetV3Large Keras モデルを利用した基本的な使用方法

この API は MobileNetV3Large などの大型のモデルでも動作します。出力が大きくなるため、任意のテキストエディタで表示した方がよい場合があります。

In [ ]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

## GPU デリゲート互換性の確認

ModelAnalyzer API には `gpu_compatibility=True` オプションがあり、特定のモデルの [GPU デリゲート](https://www.tensorflow.org/lite/performance/gpu)互換性を確認することができます。


### 事例 1: モデルの互換性がないとき

次のコードでは、シンプルな tf.function で `gpu_compatibility=True` オプションを使用する方法を示します。ここでは、2D テンソルの `tf.slice` と、GPU デリゲートに対応していない `tf.cosh` が使用されています。

互換性の問題が生じているすべてのノードで `GPU COMPATIBILITY WARNING` が表示されます。

In [ ]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

### 事例 1: モデルの互換性があるとき

この例では、特定のモデルが GPU デリゲートに対応しています。

**注意:** ツールでは互換性の問題が検出されませんが、モデルがすべてのデバイスで GPU デリゲートに対応することは保証されません。このため、ターゲット OpenGL バックエンドで `CL_DEVICE_IMAGE_SUPPORT` 機能が見つからないといった、何らかのランタイム非互換性の問題が発生する可能性があります。


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)